In [9]:
from minio import Minio
# Create client with access key and secret key with specific region.
client = Minio(
    "localhost:9000",
    access_key="1DEk417EwJy76U7FrDzQ",
    secret_key="yWWN4BlDeNHW6GpjlbPLH0QAcds7qeoH2DFkwzYN",
    secure=False
)



In [10]:
buckets = client.list_buckets()
for bucket in buckets:
    print(bucket.name, bucket.creation_date)

moviehub 2025-04-16 05:56:20.353000+00:00


In [12]:
objects = client.list_objects("moviehub", recursive=True)
for obj in objects:
    print(bucket.name)
    print(obj.object_name) 
    res = client.fget_object(bucket.name,obj.object_name, file_path= "./app/movieinput/"+obj.object_name)
    print(res)
    

moviehub
1.mp4
moviehub
10.mp4
moviehub
2.mp4
moviehub
3.mp4
moviehub
4.mp4
moviehub
5.mp4
moviehub
6.mp4
moviehub
7.mp4
moviehub
8.mp4
moviehub
9.mp4


In [13]:
import os
os.getcwd()

'/home/user/Documents/Multilingual-Transcriber/dags/app/experiments'

In [15]:
client.fget_object("moviehub","7.mp4", file_path= "/home/user/Documents/Multilingual-Transcriber/dags/app/movieinput")

In [12]:
client

In [7]:
lst = []
for i in range (5):
    lst.append([0,i])
lst

[[0, 0], [0, 1], [0, 2], [0, 3], [0, 4]]

In [12]:
for i in range(len(lst)):
    print(lst[i][1])

0
1
2
3
4


In [1]:
prompt = """ 
This is an audio recording from a classic Indian Hindi movie, likely from the 1950s to 1980s.
The audio may contain soft or degraded vocal tracks, background music, dramatic pauses, 
and cultural expressions unique to old Hindi cinema. Please transcribe all spoken Hindi dialogue clearly, 
including poetic or formal speech, song lyrics, and emotionally expressive tones. Maintain speaker changes where possible 
and try to distinguish between characters even without named diarization. 
Filter out or ignore irrelevant background noise, but preserve any meaningful non-verbal vocal expressions 
(e.g., sighs, gasps). Output should be clean and suitable for translation into modern spoken languages. """
    


In [ ]:
from openai import OpenAI
import datetime

# Step 1: Get verbose JSON response with word-level timestamps
client = OpenAI(api_key = " ")
audio_file = open("/home/csc/Documents/Multilingual-Transcriber/shared_data/movieslist/Ahista/audiofiles/ahista_ahista_part12_audio.mp3", "rb")

transcription = client.audio.transcriptions.create(
    file=audio_file,
    model="whisper-1",
    response_format="verbose_json",
    timestamp_granularities=["word"]
)

words = transcription.words  # List of word dicts with start and end times

# Step 2: Helper function to format seconds to SRT time (HH:MM:SS,mmm)
def format_srt_time(seconds):
    delta = datetime.timedelta(seconds=seconds)
    return str(delta)[:-3].replace('.', ',').rjust(12, '0')

# Step 3: Group words into sentences (basic approach: group every N words or based on pause)
srt_entries = []
current_entry = []
entry_start = None
entry_end = None
max_words_per_line = 10
pause_threshold = 1.0  # seconds

for i, word in enumerate(words):
    if not current_entry:
        entry_start = word['start']
    
    current_entry.append(word['word'])
    entry_end = word['end']
    
    # End segment on max words or long pause
    next_word = words[i + 1] if i + 1 < len(words) else None
    pause = (next_word['start'] - word['end']) if next_word else 0

    if len(current_entry) >= max_words_per_line or pause > pause_threshold or next_word is None:
        text = ' '.join(current_entry).strip()
        srt_entries.append({
            "index": len(srt_entries) + 1,
            "start": format_srt_time(entry_start),
            "end": format_srt_time(entry_end),
            "text": text
        })
        current_entry = []

# Step 4: Write to .srt file
with open("transcription.srt", "w", encoding="utf-8") as f:
    for entry in srt_entries:
        f.write(f"{entry['index']}\n")
        f.write(f"{entry['start']} --> {entry['end']}\n")
        f.write(f"{entry['text']}\n\n")

print("SRT file created: transcription.srt")


: 